In [19]:
import pandas as pd
import numpy as np
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


Let's load the bad film data

In [20]:
badfilm = pd.read_csv('BadFilmData.csv')
badfilm

,id,title,genres,averageRating,numVotes,releaseYear
0,tt5988370,Reis,"Biography, Drama",1.0,74166,2017
1,tt7221896,Cumali Ceber: Allah Seni Alsin,Comedy,1.0,39466,2017
2,tt13423846,321 Action,Drama,1.0,10215,2020
3,tt20255706,The Pogmentary,"Biography, Documentary, Sport",1.1,13805,2022
4,tt21403988,The Last Pharaoh,"Documentary, Drama, History",1.1,12019,2023
...,...,...,...,...,...,...
995,tt4334266,The Bad Batch,"Action, Horror, Mystery",5.2,36189,2016
996,tt1712170,Alex Cross,"Action, Crime, Mystery",5.2,35725,2012
997,tt0463872,Asterix at the Olympic Games,"Adventure, Comedy, Family",5.2,33127,2008
998,tt1731697,The Lords of Salem,"Horror, Thriller",5.2,32998,2012


We do not need id of the film for analysis
Lets change rating to int for consistency

In [21]:
badfilm.drop(['id'],axis=1,inplace=True)
badfilm['averageRating']=(badfilm['averageRating']*10).apply(np.int64)
badfilm

,title,genres,averageRating,numVotes,releaseYear
0,Reis,"Biography, Drama",10,74166,2017
1,Cumali Ceber: Allah Seni Alsin,Comedy,10,39466,2017
2,321 Action,Drama,10,10215,2020
3,The Pogmentary,"Biography, Documentary, Sport",11,13805,2022
4,The Last Pharaoh,"Documentary, Drama, History",11,12019,2023
...,...,...,...,...,...
995,The Bad Batch,"Action, Horror, Mystery",52,36189,2016
996,Alex Cross,"Action, Crime, Mystery",52,35725,2012
997,Asterix at the Olympic Games,"Adventure, Comedy, Family",52,33127,2008
998,The Lords of Salem,"Horror, Thriller",52,32998,2012


Lets check all the different unique genre combinations

In [22]:
print(badfilm['genres'].value_counts())

Comedy                         46
Action, Adventure, Comedy      42
Horror, Mystery, Thriller      40
Horror, Thriller               34
Comedy, Romance                28
                               ..
Action, Adventure               1
Drama, War                      1
Comedy, Reality-TV, Romance     1
Action, Comedy, Music           1
Comedy, Drama, Fantasy          1
Name: genres, Length: 194, dtype: int64


Looking for a film which looks for a specific genre and having it in a new table without changing the original table

In [23]:
Thrilfilm=badfilm
Thrilfilm['Indexes']=Thrilfilm['genres'].str.find("Thriller")
Thrilfilm

,title,genres,averageRating,numVotes,releaseYear,Indexes
0,Reis,"Biography, Drama",10,74166,2017,-1
1,Cumali Ceber: Allah Seni Alsin,Comedy,10,39466,2017,-1
2,321 Action,Drama,10,10215,2020,-1
3,The Pogmentary,"Biography, Documentary, Sport",11,13805,2022,-1
4,The Last Pharaoh,"Documentary, Drama, History",11,12019,2023,-1
...,...,...,...,...,...,...
995,The Bad Batch,"Action, Horror, Mystery",52,36189,2016,-1
996,Alex Cross,"Action, Crime, Mystery",52,35725,2012,-1
997,Asterix at the Olympic Games,"Adventure, Comedy, Family",52,33127,2008,-1
998,The Lords of Salem,"Horror, Thriller",52,32998,2012,8


Checking and listing only the film that have matches to the genre Thriller that was done in the last one

In [24]:
isthril=Thrilfilm[Thrilfilm['Indexes'] > 0]
isthril

,title,genres,averageRating,numVotes,releaseYear,Indexes
8,Smolensk,"Drama, Thriller",12,40233,2016,7
26,Birdemic: Shock and Terror,"Horror, Thriller",17,25843,2010,8
33,Radhe,"Action, Crime, Thriller",19,180273,2021,15
35,Race 3,"Action, Crime, Thriller",19,48996,2018,15
45,Baaghi 3,"Action, Adventure, Thriller",22,14175,2020,19
...,...,...,...,...,...,...
985,Child's Play 3,"Horror, Thriller",52,47866,1991,8
987,The Transporter Refueled,"Action, Thriller",52,45629,2015,8
989,Friday the 13th: The New Blood,"Horror, Thriller",52,43930,1988,8
991,Shutter,"Horror, Mystery, Thriller",52,38123,2008,17


We do not need to have titles and genres in our prediction , so we will only take rating (index 2) as our target and compare it to  vote count ( index 3) and release year (index 4) as anything else is not really quanifiable. 3:-1 can be read as - start from index 3 and omit the last one (indexes)

In [25]:
x=isthril.iloc[:,3:-1]
y=isthril.iloc[:,2]


In [26]:
x

,numVotes,releaseYear
8,40233,2016
26,25843,2010
33,180273,2021
35,48996,2018
45,14175,2020
...,...,...
985,47866,1991
987,45629,2015
989,43930,1988
991,38123,2008


In [27]:
y

8      12
26     17
33     19
35     19
45     22
       ..
985    52
987    52
989    52
991    52
998    52
Name: averageRating, Length: 214, dtype: int64

In [28]:
x_train , x_test , y_train,y_test=train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 0)

In [29]:
K=10
knn=KNeighborsClassifier(K)
knn.fit(x_train, y_train)
yPredSklearn=knn.predict(x_test)
print(yPredSklearn)

[39 50 46 39 39 46 51 47 48 50 52 51 50 45 51 51 42 12 48 50 42 50 47 51
 42 50 47 51 51 48 51 51 44 51 47 48 51 39 42 42 42 19 48]


The accuracy seems to be parabolic based to the amount of K , 150 and 4 have both 0,046 ,but it is highly probable that it repeats itself like y=cos(x) if the test size is 0.2 ,highest seems to be 12% accuracy every 40 
Testing k=10 with  different test size(0.1-0.9) .09 .11 .06 .02 .05 .11 .06 .06 .09 ,accuracy fluxuates ,but again is never over 12%

In [30]:
accuracy_score(y_test, yPredSklearn)

0.11627906976744186

Use Cases : Using the prediction to calcule what average score does it take for a thriller film to be considered bad
2nd Use case : Predicting future bad thriller film scores based to average vote amount and it's release year

K-fold Testing

In [31]:
A=badfilm.iloc[:,4]
B=badfilm.iloc[:,2]

In [32]:
A

0      2017
1      2017
2      2020
3      2022
4      2023
       ... 
995    2016
996    2012
997    2008
998    2012
999    2024
Name: releaseYear, Length: 1000, dtype: int64

In [33]:
B

0      10
1      10
2      10
3      11
4      11
       ..
995    52
996    52
997    52
998    52
999    52
Name: averageRating, Length: 1000, dtype: int64

In [34]:
Kf=KFold(n_splits=3, shuffle=True, random_state=42)

In [35]:
k_neighbors = 3
accuracies = []

for train_index, test_index in Kf.split(x):
    A_train, A_test = A[train_index], A[test_index]
    B_train, B_test = B[train_index], B[test_index]
    
    knn = KNeighborsClassifier(n_neighbors=k_neighbors)
    knn.fit(x_train, y_train)
    
    y_pred = knn.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f"Accuracies for each fold: {accuracies}")
print(f"Average Accuracy across all folds: {average_accuracy:.2f}")

Accuracies for each fold: [0.023255813953488372, 0.023255813953488372, 0.023255813953488372]
Average Accuracy across all folds: 0.02
